## Identifying Duplicate Questions

Over 100 million people visit Quora every month, so it's no surprise that many people ask similar (or the same) questions. Various questions with the same intent can cause people to spend extra time searching for the best answer to their question, and results in members answering multiple versions of the same question. Quora uses random forest to identify duplicated questions to provide a better experience to active seekers and writers, and offer more value to both of these groups in the long term.
Follow the steps outlined below to build the appropriate classifier model. 


Steps:
- Download data
- Exploration
- Cleaning
- Feature Engineering
- Modeling

By the end of this project you should have **a presentation that describes the model you built** and its **performance**. 


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../data/train.csv")

#### Note
There is no designated test.csv file. The train.csv file is the entire dataset. Part of the data in the train.csv file should be set aside to act as the final testing data.

### Exploration

In [3]:
df.shape

(404290, 6)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [8]:
df_m = df.iloc[:,3:]
df_m.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


### Cleaning

- Tokenization
- Stopwords cleaning
- Removing punctuation
- Normalizing
- Stemming

In [9]:
import string
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [10]:
lemmatizer = WordNetLemmatizer() 
lemmatizer.lemmatize('rocks')
porter = PorterStemmer()
ENGstopwords = stopwords.words('english')

In [11]:
def clean_txt(text):
    text = "".join([char for char in text if char not in string.punctuation])
    #' '.join( [w for w in text.split() if len(w)>1] )
    text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
    text = text.replace('  ', ' ')
    tokens = word_tokenize(text.lower())
    text = [word for word in tokens if word not in ENGstopwords]
    #final_text = [porter.stem(word) for word in text]
    final_text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(final_text)
    

In [12]:
df_m['question1_clean'] = df_m['question1'].apply(lambda x: clean_txt(x))

In [13]:
df_m['question2_clean'] = df_m['question2'].apply(lambda x: clean_txt(x))

In [14]:
df_m.head()

,question1,question2,is_duplicate,question1_clean,question2_clean
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math2324math divided 2423
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water


### Feature Engineering

- tf-idf
- word2vec
- word count
- number of the same words in both questions
- ....

In [15]:
df_features = df_m.iloc[:,2:]

In [16]:
df_features.head()

,is_duplicate,question1_clean,question2_clean
0,0,step step guide invest share market india,step step guide invest share market
1,0,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...
2,0,increase speed internet connection using vpn,internet speed increased hacking dns
3,0,mentally lonely solve,find remainder math2324math divided 2423
4,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water


In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_features.iloc[:,1:], df_features['is_duplicate'], test_size=0.3, random_state=42)

In [18]:
X_train

,question1_clean,question2_clean
140908,new method angioplasty cost r 5000 j hospital ...,much cost run hospital
107096,whatsapp say message info message read blue ti...,friend abroad sent message one grey tick next ...
27940,holy scripture hinduism compare contrast taoism,holy scripture hinduism compare contrast italo...
157100,long typically take get pilot license,much cost get private pilot license
111382,question havent changed marked needing improve...,question marked instantly needing improvement ...
...,...,...
259180,power positive thinking,cultivate power positive thinking
365841,thing new employee know going first day tennant,thing new employee know going first day att
131933,currently winning presidential election,bias aside point time think win presidential e...
146868,telugutamilhindi movie leading actor becomes m...,fix internet connection available problem mobi...


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
def create_doc_term_matrix(text):
    vectorizer = TfidfVectorizer(max_features = 1000)
    doc_term_matrix = vectorizer.fit_transform(text)
    return pd.DataFrame(doc_term_matrix.toarray(), columns = vectorizer.get_feature_names_out()), vectorizer

In [21]:
q1_df, vectorizer1 = create_doc_term_matrix(X_train['question1_clean'])

In [22]:
vectorizer1

TfidfVectorizer(max_features=1000)

In [23]:
q1_df.head(),

(    10  100  1000   12   15   20  2000  2015  2016  2017  ...  written  wrong  \
 0  0.0  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  ...      0.0    0.0   
 1  0.0  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  ...      0.0    0.0   
 2  0.0  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  ...      0.0    0.0   
 3  0.0  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  ...      0.0    0.0   
 4  0.0  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  ...      0.0    0.0   
 
    yahoo  year  yes  yet  young  youre  youtube  youve  
 0    0.0   0.0  0.0  0.0    0.0    0.0      0.0    0.0  
 1    0.0   0.0  0.0  0.0    0.0    0.0      0.0    0.0  
 2    0.0   0.0  0.0  0.0    0.0    0.0      0.0    0.0  
 3    0.0   0.0  0.0  0.0    0.0    0.0      0.0    0.0  
 4    0.0   0.0  0.0  0.0    0.0    0.0      0.0    0.0  
 
 [5 rows x 1000 columns],)

In [24]:
q2_df, vectorizer2 = create_doc_term_matrix(X_train['question2_clean'])
q2_df.head()

,10,100,1000,12,12th,15,16,20,2000,2015,...,written,wrong,yahoo,year,yes,yet,york,youre,youtube,youve
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
X_train_vectorized = pd.merge(q1_df,q2_df, left_index=True, right_index=True)

In [26]:
X_train_vectorized.head()

,10_x,100_x,1000_x,12_x,15_x,20_x,2000_x,2015_x,2016_x,2017_x,...,written_y,wrong_y,yahoo_y,year_y,yes_y,yet_y,york,youre_y,youtube_y,youve_y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
X_test.head()

,question1_clean,question2_clean
8067,play pokémon go korea,play pokémon go china
224279,breathing treatment help cough,help someone unconscious still breathing
252452,kellyanne conway annoying opinion,kellyanne conway really imply pay attention wo...
174039,rate 110 review maruti baleno,career option one completing bachelor degree d...
384863,good book marketing,best book ever written marketing


In [36]:
def create_test_doc_term_matrix(text, vectorizer):
    doc_term_test_matrix = vectorizer.transform(text)
    return pd.DataFrame(doc_term_test_matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [37]:
X_test_q1 = create_test_doc_term_matrix(X_test['question1_clean'], vectorizer1)

In [38]:
X_test_q2 = create_test_doc_term_matrix(X_test['question2_clean'], vectorizer2)

In [39]:
X_test_vectorized = pd.merge(X_test_q1,X_test_q2, left_index=True, right_index=True)

In [40]:
X_test_vectorized.head()

,10_x,100_x,1000_x,12_x,15_x,16,20_x,2000_x,2015_x,2016_x,...,written_y,wrong_y,yahoo,year_y,yes_y,yet,york,youre_y,youtube_y,youve_y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.585021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Modeling

Different modeling techniques can be used:

- logistic regression
- XGBoost
- LSTMs
- etc

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [41]:
clf = GaussianNB()
clf.fit(X_train_vectorized, y_train)

GaussianNB()

In [42]:
y_pred = clf.predict(X_test_vectorized)

/Users/jibuthomas/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 18
- 2014
- 25
- acid_x
- acid_y
- ...
Feature names seen at fit time, yet now missing:
- 12th
- according_x
- according_y
- acid
- adult
- ...

  warnings.warn(message, FutureWarning)


In [43]:
f1 = f1_score(y_test, y_pred)

In [44]:
f1

0.3667679138478531